In [1]:
import queue
import collections

In [2]:
def read_file(filename):
    with open(filename) as f:
        return [list(map(int, line.strip())) for line in f]

In [3]:
# lines = read_file("../input/day17-sample1.txt")
# lines = read_file("../input/day17-sample2.txt")
lines = read_file("../input/day17-input.txt")

In [4]:
# lines
len(lines), len(lines[0])

(141, 141)

In [5]:
u, d, l, r = ('u', -1, 0), ('d', 1, 0), ('l', 0, -1), ('r', 0, 1)

directions = {
    'u': [u, l, r],
    'd': [d, l, r],
    'l': [u, d, l],
    'r': [u, d, r]
}

def ingrid(i, j, height, width):
    return i >= 0 and i < height and j >= 0 and j < width

# node has form of (i, j, direction_from, steps_left)
def build_graph(lines, min_steps=0, max_steps=3):
    height = len(lines)
    width = len(lines[0])
    G = {}
    
    for i in range(height):
        for j in range(width):
            for dir_from in directions.keys():
                for steps in range(max_steps):
                    node = (i, j, dir_from, steps)
                    neigh = [
                        (i + di, j + dj, fr, steps - 1 if fr == dir_from else max_steps - 1)
                        for (fr, di, dj) in directions[dir_from]
                        if ingrid(i + di, j + dj, height, width)
                        and (steps > 0 or fr != dir_from)
                        and ((max_steps - steps) >= min_steps or fr == dir_from)
                    ]
                    G[node] = [(n, lines[n[0]][n[1]]) for n in neigh]    
    return G

In [6]:
def dijkstra(G, start):
    dist = collections.defaultdict(lambda: 1_000_000_000)
    visited = set()
    prev = dict()
    Q = queue.PriorityQueue()
    
    for s in start:
        dist[s] = 0
        Q.put((0, s))

    while Q:
        while not Q.empty():
            _, node = Q.get()
            if node not in visited:
                break
        else:
            break
            
        for (n, d) in G[node]:
            new_dist = dist[node] + d
            if new_dist < dist[n]:
                dist[n] = new_dist
                prev[n] = node
                Q.put((new_dist, n))
    return dist, prev

In [7]:
G = build_graph(lines)

In [8]:
start = [(0, 0, 'u', 2), (0, 0, 'l', 2)]
dist, prev = dijkstra(G, start)

In [9]:
part1 = min(v for k, v in dist.items() if k[0] == len(lines) - 1 and k[1] == len(lines[0]) - 1)
part1

936

In [10]:
G = build_graph(lines, min_steps=4, max_steps=10)
start = [(0, 0, 'u', 2), (0, 0, 'l', 2)]
dist, prev = dijkstra(G, start)

In [11]:
part2 = min(
    v for k, v in dist.items()
    if k[0] == len(lines) - 1 and k[1] == len(lines[0]) - 1 and k[3] <= 6
)
part2

1157

In [12]:
# [(k, v) for k, v in dist.items() if k[0] == len(lines) - 1 and k[1] == len(lines[0]) - 1 and k[3] <= 6]

In [13]:
# p = (4, 11, 'r', 9)
# path_backwards = []
# while p in prev:
#     path_backwards.append((p, dist[p]))
#     p = prev[p]
# path = path_backwards[::-1]
# path

In [14]:
# G[(0, 1, 'r', 9)]
# G[(0, 3, 'r', 8)]
# G[(0, 3, 'r', 7)]
# G[(0, 4, 'r', 6)]
# G[(0, 5, 'r', 5)]
# G[(1, 5, 'd', 9)]
# G[(2, 5, 'd', 8)]
# G[(3, 5, 'd', 7)]
# G[(4, 5, 'd', 6)]